# Walmart Deserts in USA - Poverty Rate Vs. Walmart Locations
---
The below script explores the relationship between states with high poverty rates and the number of Walmart stores in those states.

In this script, we:
1. Extracted Walmart store locations from DataWorld.com
2. Retrieved data from the 2018 US Census API and Google API to show the relationship between various socioeconomic parameters and Walmart Store counts across selected zip codes. 
3. Transformed the data using Pandas, Numpy, Matplotlib, o accomplish our task.
4. Plotted the data in heatmaps using Requests, Census API, and Google API.
5. Stored the final data into PostgreSQL using SQLAlchemy

## Extract Data: From csv file into a Pandas DataFrame

In [1]:
# Dependencies
import pandas as pd

# Read the Walmart Store Locations from the CSV file (from DataWorld)
input_file_path = "walmartstorelocation.csv"
walmart_locations_df = pd.read_csv(input_file_path)

walmart_locations_df
# Output: 4654 rows × 22 columns

,name,url,street_address,city,state,zip_code,country,phone_number_1,phone_number_2,fax_1,...,email_2,website,open_hours,latitude,longitude,facebook,twitter,instagram,pinterest,youtube
0,Conway Supercenter,https://www.walmart.com/store/5/conway-ar/details,1155 Hwy 65 North,Conway,AR,72032,US,501-329-0023,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",35.108660,-92.436905,NaN,NaN,NaN,NaN,NaN
1,Sikeston Supercenter,https://www.walmart.com/store/9/sikeston-mo/de...,1303 S Main St,Sikeston,MO,63801,US,573-472-3020,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",36.857394,-89.586051,NaN,NaN,NaN,NaN,NaN
2,Tahlequah Supercenter,https://www.walmart.com/store/10/tahlequah-ok/...,2020 S Muskogee Ave,Tahlequah,OK,74464,US,918-456-8804,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",35.888765,-94.979859,NaN,NaN,NaN,NaN,NaN
3,Mountain Home Supercenter,https://www.walmart.com/store/11/mountain-home...,65 Wal Mart Dr,Mountain Home,AR,72653,US,870-492-9299,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",36.354957,-92.341026,NaN,NaN,NaN,NaN,NaN
4,Claremore Supercenter,https://www.walmart.com/store/12/claremore-ok/...,1500 S Lynn Riggs Blvd,Claremore,OK,74017,US,918-341-2765,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",36.293955,-95.627125,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4649,Pearland Neighborhood Market,https://www.walmart.com/store/7361/pearland-tx...,12631 Broadway Street,Pearland,TX,77584,US,713-570-7925,NaN,NaN,...,NaN,NaN,"monday - friday : 06:00-23:00, saturday : 06:0...",29.556350,-95.415782,NaN,NaN,NaN,NaN,NaN
4650,Bentonville Gas Station,https://www.walmart.com/store/7368/bentonville...,1300 S Walton Blvd,Bentonville,AR,72712,US,479-271-2209,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",36.357486,-94.214669,NaN,NaN,NaN,NaN,NaN
4651,Millcreek Pickup only,https://www.walmart.com/store/8855/millcreek-u...,3701 S Highland Dr,Millcreek,UT,84106,US,801-467-2882,NaN,NaN,...,NaN,NaN,"monday - friday : 08:00-20:00, saturday : 08:0...",40.691031,-111.846529,NaN,NaN,NaN,NaN,NaN
4652,Dallas Supercenter,https://www.walmart.com/store/8930/dallas-tx/d...,15757 Coit Rd,Dallas,TX,75248,US,972-235-0681,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",32.963412,-96.770294,NaN,NaN,NaN,NaN,NaN


In [2]:
walmart_locations_clean_df = walmart_locations_df.filter(["name", "latitude", "latitude","city", "state","zip_code"])
walmart_locations_clean_df 

,name,latitude,latitude,city,state,zip_code
0,Conway Supercenter,35.108660,35.108660,Conway,AR,72032
1,Sikeston Supercenter,36.857394,36.857394,Sikeston,MO,63801
2,Tahlequah Supercenter,35.888765,35.888765,Tahlequah,OK,74464
3,Mountain Home Supercenter,36.354957,36.354957,Mountain Home,AR,72653
4,Claremore Supercenter,36.293955,36.293955,Claremore,OK,74017
...,...,...,...,...,...,...
4649,Pearland Neighborhood Market,29.556350,29.556350,Pearland,TX,77584
4650,Bentonville Gas Station,36.357486,36.357486,Bentonville,AR,72712
4651,Millcreek Pickup only,40.691031,40.691031,Millcreek,UT,84106
4652,Dallas Supercenter,32.963412,32.963412,Dallas,TX,75248


In [3]:
walmart_locations_clean_df.dropna()
walmart_locations_clean_df 

,name,latitude,latitude,city,state,zip_code
0,Conway Supercenter,35.108660,35.108660,Conway,AR,72032
1,Sikeston Supercenter,36.857394,36.857394,Sikeston,MO,63801
2,Tahlequah Supercenter,35.888765,35.888765,Tahlequah,OK,74464
3,Mountain Home Supercenter,36.354957,36.354957,Mountain Home,AR,72653
4,Claremore Supercenter,36.293955,36.293955,Claremore,OK,74017
...,...,...,...,...,...,...
4649,Pearland Neighborhood Market,29.556350,29.556350,Pearland,TX,77584
4650,Bentonville Gas Station,36.357486,36.357486,Bentonville,AR,72712
4651,Millcreek Pickup only,40.691031,40.691031,Millcreek,UT,84106
4652,Dallas Supercenter,32.963412,32.963412,Dallas,TX,75248


In [4]:
# Check the datatype of the Zipcode column
walmart_locations_df.dtypes

name               object
url                object
street_address     object
city               object
state              object
zip_code            int64
country            object
phone_number_1     object
phone_number_2    float64
fax_1             float64
fax_2             float64
email_1           float64
email_2           float64
website           float64
open_hours         object
latitude          float64
longitude         float64
facebook          float64
twitter           float64
instagram         float64
pinterest         float64
youtube           float64
dtype: object

In [5]:
# Rename columns
walmart_locations_df = walmart_locations_df.rename(columns={"name": "Store Name",
                                                           "zip_code": "Zipcode"})

# Select required columns in a new dataframe
stores_df = walmart_locations_df[["Store Name", "Zipcode"]]
stores_df
# Output: 4654 rows × 2 columns

,Store Name,Zipcode
0,Conway Supercenter,72032
1,Sikeston Supercenter,63801
2,Tahlequah Supercenter,74464
3,Mountain Home Supercenter,72653
4,Claremore Supercenter,74017
...,...,...
4649,Pearland Neighborhood Market,77584
4650,Bentonville Gas Station,72712
4651,Millcreek Pickup only,84106
4652,Dallas Supercenter,75248


In [6]:
# Group by zipcode to find the count of stores per zipcode and store this grouped data in a new dataframe
store_count_df = stores_df.groupby(["Zipcode"]).count()
store_count_df
# Output: 4177 rows × 1 columns

,Store Name
Zipcode,
617,1
659,1
662,1
674,1
680,1
...,...
99611,1
99615,1
99654,1


In [7]:
# Rename the Name column to "Store Count" column 
store_count_df = store_count_df.rename(columns={'Store Name': "Store Count"})
store_count_df

,Store Count
Zipcode,
617,1
659,1
662,1
674,1
680,1
...,...
99611,1
99615,1
99654,1


In [8]:
# Reset the Index and store in a new dataframe
store_count_clean_df = store_count_df.reset_index()
store_count_clean_df

,Zipcode,Store Count
0,617,1
1,659,1
2,662,1
3,674,1
4,680,1
...,...,...
4172,99611,1
4173,99615,1
4174,99654,1
4175,99701,1


## Run Census Search to retrieve data on all zip codes (2018 ACS5 Census)
##### See: https://github.com/CommerceDataService/census-wrapper for library documentation
##### See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

In [9]:
# Census Dependencies
# Census API Key
from config import (census_key, gkey)
from census import Census
c = Census(census_key, year=2018)

census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = (census_df["Poverty Count"]/census_df["Population"])*100
row_count = len(census_df)

print(f"Total number of records: {row_count}")
census_df

# Output: 33120 rows × 4 columns

Total number of records: 33120


,Population,Poverty Count,Zipcode,Poverty Rate
0,17242.0,10772.0,00601,62.475351
1,38442.0,19611.0,00602,51.014515
2,48814.0,24337.0,00603,49.856599
3,6437.0,4163.0,00606,64.672984
4,27073.0,11724.0,00610,43.305138
...,...,...,...,...
33115,15.0,0.0,99923,0.000000
33116,927.0,172.0,99925,18.554477
33117,1635.0,235.0,99926,14.373089
33118,38.0,28.0,99927,73.684211


In [10]:
census_df.dtypes

Population       float64
Poverty Count    float64
Zipcode           object
Poverty Rate     float64
dtype: object

In [11]:
census_df["Zipcode"] = census_df["Zipcode"].astype(int)
census_df.dtypes

Population       float64
Poverty Count    float64
Zipcode            int32
Poverty Rate     float64
dtype: object

## Transform Data 

In [12]:
# Merge the two data sets along zip code
stores_per_zip_df = pd.merge(store_count_clean_df, census_df, how="left", on=["Zipcode"])

# Remove rows missing data
stores_per_zip_clean_df = stores_per_zip_df.dropna()

# Visualize
stores_per_zip_clean_df

#Output: 4168 rows × 5 columns

,Zipcode,Store Count,Population,Poverty Count,Poverty Rate
0,617,1,23974.0,11879.0,49.549512
1,659,1,40390.0,18717.0,46.340678
2,662,1,40290.0,21809.0,54.130057
3,674,1,39859.0,18226.0,45.726185
4,680,1,46395.0,23841.0,51.387003
...,...,...,...,...,...
4172,99611,1,15657.0,2041.0,13.035703
4173,99615,1,12971.0,1100.0,8.480456
4174,99654,1,62135.0,6236.0,10.036211
4175,99701,1,17510.0,2359.0,13.472302


In [17]:
stores_per_zip_clean_df.dtypes

Zipcode            int64
Store Count        int64
Population       float64
Poverty Count    float64
Poverty Rate     float64
dtype: object

# # Loading Data into a PostgreSQL Database

In [13]:
# Dependencies
from sqlalchemy import create_engine
from config import (postgres_id, postgres_pw)

In [14]:
# Connect to a local database
conn_str = f"{postgres_id}:{postgres_pw}@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{conn_str}')

In [15]:
# check tables in teh database
engine.table_names()

['store_counts', 'walmart_stores']

In [18]:
# Load the merged DataFrames into the database
stores_per_zip_clean_df.to_sql(name='walmart_stores', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Zipcode" of relation "walmart_stores" does not exist
LINE 1: INSERT INTO walmart_stores ("Zipcode", "Store Count", "Popul...
                                    ^

[SQL: INSERT INTO walmart_stores ("Zipcode", "Store Count", "Population", "Poverty Count", "Poverty Rate") VALUES (%(Zipcode)s, %(Store Count)s, %(Population)s, %(Poverty Count)s, %(Poverty Rate)s)]
[parameters: ({'Zipcode': 617, 'Store Count': 1, 'Population': 23974.0, 'Poverty Count': 11879.0, 'Poverty Rate': 49.549511971302245}, {'Zipcode': 659, 'Store Count': 1, 'Population': 40390.0, 'Poverty Count': 18717.0, 'Poverty Rate': 46.34067838573904}, {'Zipcode': 662, 'Store Count': 1, 'Population': 40290.0, 'Poverty Count': 21809.0, 'Poverty Rate': 54.130057086125596}, {'Zipcode': 674, 'Store Count': 1, 'Population': 39859.0, 'Poverty Count': 18226.0, 'Poverty Rate': 45.72618480142502}, {'Zipcode': 680, 'Store Count': 1, 'Population': 46395.0, 'Poverty Count': 23841.

In [24]:
# Load the merged DataFrames into the database
stores_per_zip_clean_df.to_sql(name='walmart_store', con=engine, if_exists='append', index=False)

In [23]:
# Load the DataFrames into the database
store_count_clean_df.to_sql(name='store_counts', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Zipcode" of relation "store_counts" does not exist
LINE 1: INSERT INTO store_counts ("Zipcode", "Store Count") VALUES (...
                                  ^

[SQL: INSERT INTO store_counts ("Zipcode", "Store Count") VALUES (%(Zipcode)s, %(Store Count)s)]
[parameters: ({'Zipcode': 617, 'Store Count': 1}, {'Zipcode': 659, 'Store Count': 1}, {'Zipcode': 662, 'Store Count': 1}, {'Zipcode': 674, 'Store Count': 1}, {'Zipcode': 680, 'Store Count': 1}, {'Zipcode': 725, 'Store Count': 1}, {'Zipcode': 728, 'Store Count': 1}, {'Zipcode': 729, 'Store Count': 1}  ... displaying 10 of 4177 total bound parameter sets ...  {'Zipcode': 99701, 'Store Count': 1}, {'Zipcode': 99901, 'Store Count': 1})]
(Background on this error at: http://sqlalche.me/e/f405)

In [22]:
pd.read_sql_query('select * from store_counts', con=engine)

,Zipcode,Store Count
0,617,1
1,659,1
2,662,1
3,674,1
4,680,1
...,...,...
4172,99611,1
4173,99615,1
4174,99654,1
4175,99701,1


## Heatmap of Poverty Rate

In [33]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt


In [34]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [35]:
# Store the Walmart store location (Latitude and 'Longitude) into  a series "locations" for plotting
locations = walmart_locations_df[["latitude", "longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
# walmart_locations_df = walmart_locations_df.dropna()
poverty_rate = stores_per_zip_clean_df["Poverty Rate"].astype(float)

In [36]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

ValueError: weights must be of the same length as locations or None